In [1]:
'''
Import packages and display settings
'''
## supress warnings
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
## display settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('max_info_columns', 10000000)
import os
## Assign dataset path
path_data = 'C:/Users/Cody_Black/JupyterNotebook/Dataset'
os.chdir(path_data)

In [2]:
'''
Load data sets
'''

df_A_Test = pd.read_excel(open('Babson Hackathon _ Exhibits A _ B.xlsx', 'rb'), sheet_name='Exhibit A | Test Results') 
# df_B_Content = pd.read_excel(open('Babson Hackathon _ Exhibits A _ B.xlsx', 'rb'), sheet_name='Exhibit B | Content Type Defini')  
df_ContentRecomm = pd.read_csv('ContentRecommendationData.csv')
df_ModuleCount = pd.read_csv('ModuleCountData.csv')
# df_Supplier = pd.read_csv('SupplierData.csv')
# df_Supplier_UK = pd.read_csv('SupplierData_Wayfair UK.csv')
df_Supplier_US = pd.read_csv('SupplierData_Wayfair.csv')

In [3]:
'''
Revise column names
'''
for df in [df_A_Test, df_ContentRecomm, df_ModuleCount, df_Supplier_US]:
    df.columns = df.columns.str.replace(" ", "_")


## Add prefix to columns in each df
df_Supplier_US = df_Supplier_US.add_prefix('SP_')
df_ContentRecomm = df_ContentRecomm.add_prefix('CR_')
df_ModuleCount = df_ModuleCount.add_prefix('MC_')
df_A_Test = df_A_Test.add_prefix('AT_')

In [4]:
'''
Merging purpose: Create class name for df_ContentRecomm
'''

### Since there is no class name in df_ContentRecomm, we use supplier data set to fill in class name by using clid as keys
df_Supplier = pd.read_csv('SupplierData.csv')
df_Supplier = df_Supplier.add_prefix('SP_')
df_Supplier_subset = df_Supplier[['SP_clid', 'SP_clname']]
df_Supplier_subset.drop_duplicates(subset =['SP_clid', 'SP_clname'],
                                      keep = 'first', inplace = True) 
df_merge = pd.merge(df_ContentRecomm, df_Supplier_subset,
                   how='left',
                   left_on='CR_clid', right_on='SP_clid')
# print('Check if there is any row in CR can not match with SP: ')
# print(df_merge[df_merge['SP_clname'].isnull()])
df_ContentRecomm['CR_clname'] = df_merge['SP_clname']
del df_Supplier, df_Supplier_subset, df_merge

'''
Re-order columns
'''
# cols = df_ContentRecomm.columns.tolist()
cols= ['CR_prsku',
       'CR_clid',
       'CR_clname',
       'CR_percentilerank',
       'CR_biccontenttypename',
       'CR_hasbiccontenttype']
df_ContentRecomm = df_ContentRecomm[cols]

<code>Check if there is any row in CR can not match with SP: 
Empty DataFrame
Columns: [CR_prsku, CR_clid, CR_percentilerank, CR_biccontenttypename, CR_hasbiccontenttype, SP_clid, SP_clname]
Index: []</code>

In [5]:
'''
recast types
'''

for col in ['AT_clmkcid']:
    df_A_Test[col] = df_A_Test[col].astype('category')
for col in ['CR_prsku', 'CR_clid', 'CR_hasbiccontenttype']:
    df_ContentRecomm[col] = df_ContentRecomm[col].astype('category')
for col in ['MC_prsku', 'MC_clid']:
    df_ModuleCount[col] = df_ModuleCount[col].astype('category')
for col in ['SP_soid', 'SP_clid', 'SP_suid']: 
    df_Supplier_US[col] = df_Supplier_US[col].astype('category')
# df_Supplier_US['SP_iscurrent'] = df_Supplier_US['SP_iscurrent'].astype('int')

<code>array(['Bedding Sets', 'Vanities', 'Outdoor Conversation Sets',
       'End Tables', 'Area Rugs', 'Cribs', 'Kids Dressers & Chests',
       'Wall Art', 'TV Stands & Entertainment Centers', 'Headboards',
       'Accent Chests / Cabinets', 'Desks', 'Sofas', 'Dressers & Chests',
       'Bar Stools', 'Dining Table Sets', 'Garage Storage Cabinets',
       'Beds', 'Chandeliers', 'Air Fryers', 'Mattress Toppers and Pads',
       'Classroom Storage', 'Adjustable Beds', 'Pool Tables',
       'Filing Cabinets', 'Outdoor Fireplaces', 'Bathroom Storage',
       'Kitchen Islands', 'Interior Doors', 'Reception Seating Chairs',
       'Ceiling Fans', 'Gliders', 'Adirondack Chairs', 'Patio Sofas',
       'Kids Beds', 'Pantry Cabinets', 'Charcoal Grills',
       'Wine Refrigerators', 'Tubs And Whirlpools',
       'Innerspring Mattresses', 'Gas Grills',
       'Cat Litter Boxes & Litter Box Enclosures', 'Ranges', 'Mantels',
       'Electric Grills', 'Swing Sets & Playgrounds', 'Smokers',
       'Wood Pellet Grills'], dtype=object)</code>

In [6]:
'''
Filtering for analysis purpose
'''
### Copy DataFrame
df_Supplier_US_m = df_Supplier_US
df_ContentRecomm_m = df_ContentRecomm
df_A_Test_m = df_A_Test
df_ModuleCount_m = df_ModuleCount

### Drop class id since it can not match with class in in supplier data set 
df_A_Test_m = df_A_Test_m.drop(columns='AT_clid')

### Filter out suid = 1, since it is aggregation of the other suppliers
## Save it to new dataframe which uses _m as modified
df_Supplier_US_m = df_Supplier_US_m[df_Supplier_US_m['SP_suid']!=1]

### Select rows with is_current = 1 where is our current interest target
df_Supplier_US_m = df_Supplier_US_m[df_Supplier_US_m['SP_iscurrent']==1]

### Select product under class TV Stands & Entertainment Centers, since it is
# a simpler component compared to other classes
# df_Supplier_US_m = df_Supplier_US_m[df_Supplier_US_m['SP_clname']=='TV Stands & Entertainment Centers']
# df_ContentRecomm_m = df_ContentRecomm_m[df_ContentRecomm_m['CR_clname']=='TV Stands & Entertainment Centers']
# df_A_Test_m = df_A_Test_m[df_A_Test_m['AT_clname']=='TV Stands & Entertainment Centers']
# df_ModuleCount_m = df_ModuleCount_m[df_ModuleCount_m['MC_clname']=='TV Stands & Entertainment Centers']

### Only select products including in df_Supplier_US data set 
df_ContentRecomm_m = df_ContentRecomm_m[df_ContentRecomm_m['CR_prsku'].isin(
    set(df_Supplier_US_m['SP_prsku'].values).intersection( set(df_ContentRecomm_m['CR_prsku'].values) ) 
)]
df_ModuleCount_m = df_ModuleCount_m[df_ModuleCount_m['MC_prsku'].isin(
    set(df_Supplier_US_m['SP_prsku'].values).intersection( set(df_ModuleCount_m['MC_prsku'].values) ) 
)]

In [7]:
'''
Check each shape of data sets
including all classes
'''

[df_Supplier_US_m.shape, df_ContentRecomm_m.shape, df_ModuleCount_m.shape, df_A_Test_m.shape]

[(501758, 20), (1656328, 6), (80872, 5), (192, 5)]

<code>[(7344, 20), (22248, 6), (491, 5), (4, 5)]<code/>

<h3>Merge all data sets

<h5>Merge df_ContentRecomm with df_A_Test</h5>
    
First, let's first see few rows from each dataframe

In [18]:
pd.concat([df_ContentRecomm_m.sort_values(['CR_clname', 'CR_prsku', 'CR_biccontenttypename']).head(4),
          df_A_Test_m.sort_values(['AT_clname', 'AT_biccontenttypename']).head(4)])

,CR_prsku,CR_clid,CR_clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_clname,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,AT_mkcname
180613,AAD10412,14,Accent Chests / Cabinets,0.95,Dimensions,0,NaN,NaN,NaN,NaN,NaN
119869,AAD10412,14,Accent Chests / Cabinets,0.95,Materials / How Its Made,0,NaN,NaN,NaN,NaN,NaN
171013,AAD10412,14,Accent Chests / Cabinets,0.95,Relative Size & Fit,0,NaN,NaN,NaN,NaN,NaN
278470,AAD10412,14,Accent Chests / Cabinets,0.95,Visual Details,0,NaN,NaN,NaN,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,Accent Chests / Cabinets,Dimensions,-0.01,116,Accent Furniture
53,NaN,NaN,NaN,NaN,NaN,NaN,Accent Chests / Cabinets,Materials / How Its Made,-0.01,116,Accent Furniture
54,NaN,NaN,NaN,NaN,NaN,NaN,Accent Chests / Cabinets,Relative Size & Fit,0.01,116,Accent Furniture
55,NaN,NaN,NaN,NaN,NaN,NaN,Accent Chests / Cabinets,Visual Details,0.07,116,Accent Furniture


In [23]:
'''
Research on df_ContentRecomm_m:

We could see that the contents in each class does not affect by different products.
Namely, each class will only have the same 4 contents.
'''
df_ContentRecomm_m.groupby(['CR_clname', 'CR_biccontenttypename']).count()

CR_prsku  \
CR_clname                                CR_biccontenttypename                       
Accent Chests / Cabinets                 Dimensions                           3600   
                                         Materials / How Its Made             3600   
                                         Relative Size & Fit                  3600   
                                         Visual Details                       3600   
Adirondack Chairs                        Assembly & Installation               493   
                                         Dimensions                            493   
                                         Materials / How Its Made              493   
                                         Weight                                493   
Adjustable Beds                          Compatibility & Adjustability          73   
                                         Dimensions                             73   
                                         Product Feature                        73   
                                         What's In the Box                      73   
Air Fryers                               Cleaning & Care                       235   
                                         Product Capacity                      235   
                                         Product Feature                       235   
                                         What's In the Box                     235   
Area Rugs                                Cleaning & Care                     82079   
                                         Materials / How Its Made            82079   
                                         Relative Size & Fit                 82079   
                                         Warranty & Guarantees               82079   
Bar Stools                               Dimensions                           9745   
                                         Materials / How Its Made             9745   
                                         Relative Size & Fit                  9745   
                                         What's In the Box                    9745   
Bathroom Storage                         Dimensions                           1394   
                                         Materials / How Its Made             1394   
                                         Relative Size & Fit                  1394   
                                         Visual Details                       1394   
Bedding Sets                             Materials / How Its Made            23164   
                                         Product Feature                     23164   
                                         Touch & Feel                        23164   
                                         What's In the Box                   23164   
Beds                                     Compatibility & Adjustability        7329   
                                         Dimensions                           7329   
                                         Materials / How Its Made             7329   
                                         Product Feature                      7329   
Cat Litter Boxes & Litter Box Enclosures Cleaning & Care                       143   
                                         Dimensions                            143   
                                         Product Feature                       143   
                                         Relative Size & Fit                   143   
Ceiling Fans                             Assembly & Installation              2134   
                                         Dimensions                           2134   
                                         Product Feature                      2134   
                                         What's In the Box                    2134   
Chandeliers                              Assembly & Installation             15660   
                                

In [24]:
'''
Research on df_A_Test_m:

Again, we could see that each class will only have the same 4 contents.
'''
df_A_Test_m.groupby(['AT_clname', 'AT_biccontenttypename']).count()

AT_Conversion_Rate_Change  \
AT_clname                                AT_biccontenttypename                                        
Accent Chests / Cabinets                 Dimensions                                               1   
                                         Materials / How Its Made                                 1   
                                         Relative Size & Fit                                      1   
                                         Visual Details                                           1   
Adirondack Chairs                        Assembly & Installation                                  1   
                                         Dimensions                                               1   
                                         Materials / How Its Made                                 1   
                                         Weight                                                   1   
Adjustable Beds                          Compatibility & Adjustability                            1   
                                         Dimensions                                               1   
                                         Product Feature                                          1   
                                         What's In the Box                                        1   
Air Fryers                               Cleaning & Care                                          1   
                                         Product Capacity                                         1   
                                         Product Feature                                          1   
                                         What's In the Box                                        1   
Area Rugs                                Cleaning & Care                                          1   
                                         Materials / How Its Made                                 1   
                                         Relative Size & Fit                                      1   
                                         Warranty & Guarantees                                    1   
Bar Stools                               Dimensions                                               1   
                                         Materials / How Its Made                                 1   
                                         Relative Size & Fit                                      1   
                                         What's In the Box                                        1   
Bathroom Storage                         Dimensions                                               1   
                                         Materials / How Its Made                                 1   
                                         Relative Size & Fit                                      1   
                                         Visual Details                                           1   
Bedding Sets                             Materials / How Its Made                                 1   
                                         Product Feature                                          1   
                                         Touch & Feel                                             1   
                                         What's In the Box                                        1   
Beds                                     Compatibility & Adjustability                            1   
                                         Dimensions                                               1   
                                         Materials / How Its Made                                 1   
                                         Product Feature                                          1   
Cat Litter Boxes & Litter Box Enclosures Cleaning & Care                                          1   
                                         Dimensions      

In [63]:
'''
Check how many unique values in each column
'''
df_ContentRecomm_m.nunique(dropna=True)

CR_prsku                 414082
CR_clid                      48
CR_clname                    48
CR_percentilerank            98
CR_biccontenttypename        17
CR_hasbiccontenttype          2
dtype: int64

In [64]:
df_A_Test_m.nunique(dropna=True)

AT_clname                    48
AT_biccontenttypename        17
AT_Conversion_Rate_Change    11
AT_clmkcid                   26
AT_mkcname                   26
dtype: int64

In [88]:
'''
Merge df_ContentRecomm with df_A_Test
'''

df_CR_AT = pd.merge(df_ContentRecomm_m.rename(columns={'CR_clname':'clname'}),
                    df_A_Test_m.rename(columns={'AT_clname':'clname'}),
                    how = 'outer',
                    left_on = ['clname', 'CR_biccontenttypename'], 
                    right_on = ['clname', 'AT_biccontenttypename']).sort_values(by=['clname', 'CR_prsku'])

<h5>The contents under df_ContentRecomm are the same as df_A_Test

In [89]:
'''
Check if there is any null value in AT_biccontentypename.
Namely, if there's any content under any class in df_A_Test could not match with df_ContentRecomm.
In other words, the contents of AB testing in each class may different from the contents recommended by Wayfair customer research team
'''
df_CR_AT[df_CR_AT['AT_biccontenttypename'].isnull()]

,CR_prsku,CR_clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,AT_mkcname


In [27]:
'''
We could also use info function to check null value.
Other than column CR_percentilerank, there is no any null value in other columns
'''
# df_CR_AT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1656328 entries, 772455 to 1656326
Data columns (total 10 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   CR_prsku                   1656328 non-null  category
 1   CR_clid                    1656328 non-null  category
 2   clname                     1656328 non-null  object  
 3   CR_percentilerank          1121784 non-null  float64 
 4   CR_biccontenttypename      1656328 non-null  object  
 5   CR_hasbiccontenttype       1656328 non-null  category
 6   AT_biccontenttypename      1656328 non-null  object  
 7   AT_Conversion_Rate_Change  1656328 non-null  float64 
 8   AT_clmkcid                 1656328 non-null  category
 9   AT_mkcname                 1656328 non-null  object  
dtypes: category(4), float64(2), object(4)
memory usage: 194.1+ MB


In [38]:
'''
See few rows from the merging result
'''
df_CR_AT.iloc[[0,20000,500000], :]

,CR_prsku,CR_clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,AT_mkcname
772455,AAD10412,14,Accent Chests / Cabinets,0.95,Visual Details,0,Visual Details,0.07,116,Accent Furniture
38791,ACHM1506,15,Area Rugs,NaN,Warranty & Guarantees,0,Warranty & Guarantees,0.00,28,Rugs
816736,LDER2492,2,Beds,0.98,Dimensions,0,Dimensions,0.05,61,Furniture - Bedroom


In [62]:
'''
Check how many unique values in each column in order to inspect the merging result
'''
df_CR_AT.nunique(dropna=True)

CR_prsku                     414082
CR_clid                          48
clname                           48
CR_percentilerank                98
CR_biccontenttypename            17
CR_hasbiccontenttype              2
AT_biccontenttypename            17
AT_Conversion_Rate_Change        11
AT_clmkcid                       26
AT_mkcname                       26
dtype: int64

<h5>Next, merge df_CR_AT with df_ModuleCount</h5>
    
Again, let's see a row from each dataframe

In [39]:
pd.concat([df_CR_AT.head(1),
          df_ModuleCount_m.head(1)])

,CR_prsku,CR_clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,AT_mkcname,MC_prsku,MC_modulecount,MC_clid,MC_clname,MC_mkcname
772455,AAD10412,14,Accent Chests / Cabinets,0.95,Visual Details,0,Visual Details,0.07,116,Accent Furniture,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAOJ1497,1.0,34,Wall Art,Wall Art


In [40]:
'''
Since the unique products in df_ContentRecomm_m is same as df_Supplier_US_m, and
we filter out products only including in df_Supplier_US_m for df_ModuleCount_m,
therefore, we could say that the products in df_ModuleCount_m would also in df_ContentRecomm_m
(which means we can merge df_ContentRecomm_m and df_ModuleCount_m by prsku.)
'''
[df_Supplier_US_m.SP_prsku.nunique(), df_ContentRecomm_m.CR_prsku.nunique(), df_ModuleCount_m.MC_prsku.nunique()]

[414082, 414082, 80872]

In [43]:
'''
Check null values in df_ModuleCount_m:
There is no null value in it.
'''
df_ModuleCount_m.info()
df_CR_AT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80872 entries, 0 to 196909
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   MC_prsku        80872 non-null  category
 1   MC_modulecount  80872 non-null  int64   
 2   MC_clid         80872 non-null  category
 3   MC_clname       80872 non-null  object  
 4   MC_mkcname      80872 non-null  object  
dtypes: category(2), int64(1), object(2)
memory usage: 9.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1656328 entries, 772455 to 1656326
Data columns (total 10 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   CR_prsku                   1656328 non-null  category
 1   CR_clid                    1656328 non-null  category
 2   clname                     1656328 non-null  object  
 3   CR_percentilerank          1121784 non-null  float64 
 4   CR_biccontenttypename      16

<h4>For the columns <code>prsku</code>, <code>clid</code>, <code>clname</code>, and <code>mkcname</code> in df_ModuleCount_m, their uniqe values are a subset of df_CR_AT's coresponding columns.

In [90]:
'''
Since we are going to use 'prsku', 'clid', 'clname', and 'mkcname' as our merging columns,
we first check if the unique values in df_ModuleCount_m of each column are included in df_CR_AT
'''
[len(set(df_ModuleCount_m['MC_prsku'].values)) - len(set(df_CR_AT['CR_prsku'].values).intersection(set(df_ModuleCount_m['MC_prsku'].values))),
len(set(df_ModuleCount_m['MC_clid'].values)) - len(set(df_CR_AT['CR_clid'].values).intersection(set(df_ModuleCount_m['MC_clid'].values))),
len(set(df_ModuleCount_m['MC_clname'].values)) - len(set(df_CR_AT['clname'].values).intersection(set(df_ModuleCount_m['MC_clname'].values))),
len(set(df_ModuleCount_m['MC_mkcname'].values)) - len(set(df_CR_AT['AT_mkcname'].values).intersection(set(df_ModuleCount_m['MC_mkcname'].values))),]

[0, 0, 0, 0]

In [95]:
'''
Merge df_CR_AT with df_ModuleCount
'''

df_CR_MC_AT = pd.merge(df_CR_AT.rename(columns={'CR_prsku': 'prsku', 'CR_clid': 'clid', 'AT_mkcname':'mkcname'}),
                    df_ModuleCount_m.rename(columns={'MC_prsku': 'prsku', 'MC_clid': 'clid', 'MC_clname':'clname', 'MC_mkcname': 'mkcname'}),
                    how = 'left',
                    on = ['prsku', 'clid', 'clname', 'mkcname']).sort_values(by=['prsku', 'mkcname', 'clname'])

In [59]:
'''
Check merging result
'''
# df_CR_MC_AT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1656328 entries, 17604 to 390475
Data columns (total 11 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   prsku                      1656328 non-null  object  
 1   clid                       1656328 non-null  category
 2   clname                     1656328 non-null  object  
 3   CR_percentilerank          1121784 non-null  float64 
 4   CR_biccontenttypename      1656328 non-null  object  
 5   CR_hasbiccontenttype       1656328 non-null  category
 6   AT_biccontenttypename      1656328 non-null  object  
 7   AT_Conversion_Rate_Change  1656328 non-null  float64 
 8   AT_clmkcid                 1656328 non-null  category
 9   mkcname                    1656328 non-null  object  
 10  MC_modulecount             323488 non-null   float64 
dtypes: category(3), float64(3), object(5)
memory usage: 118.5+ MB


In [60]:
'''
See few rows from the merging result
'''
df_CR_MC_AT.iloc[[0,5,9], :]

,prsku,clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,mkcname,MC_modulecount
17604,AACE1000,15,Area Rugs,NaN,Warranty & Guarantees,0,Warranty & Guarantees,0.00,28,Rugs,NaN
390477,AACX1019,33,Bedding Sets,0.78,Materials / How Its Made,0,Materials / How Its Made,0.03,42,Bedding,NaN
390481,AACX1020,33,Bedding Sets,0.30,Materials / How Its Made,0,Materials / How Its Made,0.03,42,Bedding,NaN


In [69]:
'''
In this case, we don't have contents that are not matching together between CR and AT data sets,
however, in other cases, we might want to check the null value in AT_biccontenttypename to see
if there are un-matching contents.
'''

pd.concat([df_CR_MC_AT[df_CR_MC_AT['AT_biccontenttypename'].notnull()].head(4),
           df_CR_MC_AT[df_CR_MC_AT['AT_biccontenttypename'].isnull()].head(4)])

,prsku,clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,MC_modulecount
0,AAGE1000,1,TV Stands & Entertainment Centers,NaN,Relative Size & Fit,1,Relative Size & Fit,0.05,7,4.0
1,AAGE1000,1,TV Stands & Entertainment Centers,NaN,Materials / How Its Made,0,Materials / How Its Made,0.03,7,4.0
2,AAGE1000,1,TV Stands & Entertainment Centers,NaN,Product Capacity,0,Product Capacity,-0.01,7,4.0
3,AAGE1000,1,TV Stands & Entertainment Centers,NaN,Visual Details,0,Visual Details,0.02,7,4.0


<code>True    1964
dtype: int64</code>

In [61]:
'''
Check how many unique values in each column in order to inspect the merging result
'''
df_CR_MC_AT.nunique(dropna=True)

prsku                        414082
clid                             48
clname                           48
CR_percentilerank                98
CR_biccontenttypename            17
CR_hasbiccontenttype              2
AT_biccontenttypename            17
AT_Conversion_Rate_Change        11
AT_clmkcid                       26
mkcname                          26
MC_modulecount                   62
dtype: int64

<h5>Next, merge df_Supplier_US with df_CR_MC_AT</h5>
    
Again, let's first see a row from each dataframe

In [65]:
pd.concat([df_Supplier_US_m.head(1),
          df_CR_MC_AT.head(1)])

,SP_prsku,SP_soid,SP_soname,SP_clid,SP_clname,SP_mkcname,SP_suid,SP_iswaymore,SP_addedtocart,SP_placedorder,SP_trafficcount,SP_iscurrent,SP_videocount,SP_grs1month,SP_grs2month,SP_grs3month,SP_grs12month,SP_weightedavgscore,SP_percentilerank,SP_expectedgrs,prsku,clid,clname,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,AT_clmkcid,mkcname,MC_modulecount
9,AADZ1368,1,Wayfair,15,Area Rugs,Rugs,2103,1.0,1.0,0.0,114.0,1.0,0.0,0.0,2105.571894,2105.571894,7851.547682,1.22,0.79,807.8266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AACE1000,15,Area Rugs,NaN,Warranty & Guarantees,0,Warranty & Guarantees,0.0,28,Rugs,NaN


In [66]:
'''
Check null values
'''
# df_Supplier_US_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501758 entries, 9 to 3780274
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   SP_prsku             501758 non-null  object  
 1   SP_soid              501758 non-null  category
 2   SP_soname            501758 non-null  object  
 3   SP_clid              501758 non-null  category
 4   SP_clname            501758 non-null  object  
 5   SP_mkcname           501758 non-null  object  
 6   SP_suid              501758 non-null  category
 7   SP_iswaymore         501758 non-null  int64   
 8   SP_addedtocart       501758 non-null  int64   
 9   SP_placedorder       501758 non-null  int64   
 10  SP_trafficcount      501758 non-null  int64   
 11  SP_iscurrent         501758 non-null  int64   
 12  SP_videocount        501758 non-null  int64   
 13  SP_grs1month         501758 non-null  float64 
 14  SP_grs2month         501758 non-null  float64 
 15 

In [67]:
'''
Since we are going to use 'prsku', 'clid', 'clname', and 'mkcname' as our merging columns,
we first check if the unique values in df_CR_MC_AT of each column are included in df_Supplier_US_m
'''
[len(set(df_CR_MC_AT['prsku'].values)) - len(set(df_Supplier_US_m['SP_prsku'].values).intersection(set(df_CR_MC_AT['prsku'].values))),
len(set(df_CR_MC_AT['clid'].values)) - len(set(df_Supplier_US_m['SP_clid'].values).intersection(set(df_CR_MC_AT['clid'].values))),
len(set(df_CR_MC_AT['clname'].values)) - len(set(df_Supplier_US_m['SP_clname'].values).intersection(set(df_CR_MC_AT['clname'].values))),
len(set(df_CR_MC_AT['mkcname'].values)) - len(set(df_Supplier_US_m['SP_mkcname'].values).intersection(set(df_CR_MC_AT['mkcname'].values))),]

[0, 0, 0, 0]

In [96]:
'''
Merge df_Supplier_US with df_CR_MC_AT
'''

df_merge = pd.merge(df_Supplier_US_m.rename(columns={'SP_prsku':'prsku', 'SP_clid':'clid', 'SP_clname': 'clname', 'SP_mkcname': 'mkcname'}), 
                    df_CR_MC_AT,
                    how = 'left',
                    on = ['prsku', 'clid', 'clname', 'mkcname']
                   ).sort_values(by=['prsku'])

# df_merge2 = pd.merge(df_Supplier_US.rename(columns={'SP_prsku':'prsku', 'SP_clid':'clid', 'SP_clname': 'clname', 'SP_mkcname': 'mkcname'}), 
#                     df_merge.rename(columns={'AT_mkcname': 'mkcname'}),
#                    how = 'outer',
#                    on = ['prsku', 'clid', 'clname', 'mkcname']).sort_values(by=['prsku'])

In [75]:
'''
Check how many unique values in each column in order to inspect the merging result
'''

df_merge.nunique(dropna=False)

prsku                        414082
mkcname                          26
clname                           48
SP_suid                        3732
SP_iswaymore                      2
CR_percentilerank                98
CR_biccontenttypename            17
CR_hasbiccontenttype              2
AT_biccontenttypename            17
AT_Conversion_Rate_Change        11
SP_addedtocart                 1675
SP_placedorder                  518
SP_trafficcount                8035
SP_videocount                    13
MC_modulecount                   62
SP_grs1month                  70314
SP_grs2month                  99971
SP_grs3month                 122023
SP_grs12month                213628
SP_weightedavgscore             157
SP_percentilerank                98
SP_expectedgrs               117874
clid                             48
SP_iscurrent                      1
dtype: int64

In [97]:
'''
Recast data type
'''
for col in ['mkcname', 'clname', 'SP_suid', 'CR_biccontenttypename', 'CR_biccontenttypename', 'AT_biccontenttypename', 'clid']: 
    df_merge[col] = df_merge[col].astype('category')
    
for col in ['SP_iswaymore', 'CR_hasbiccontenttype', 'SP_iscurrent']: 
    df_merge[col] = df_merge[col].astype('int')

In [98]:
'''
Re order columns
'''
cols = \
['prsku',
 'mkcname',
 'clname',
 'SP_suid',
 'SP_iswaymore',
 'CR_percentilerank',
 'CR_biccontenttypename',
 'CR_hasbiccontenttype',
 'AT_biccontenttypename', 
 'AT_Conversion_Rate_Change',
 'SP_addedtocart',
 'SP_placedorder',
 'SP_trafficcount',
 'SP_videocount',
 'MC_modulecount',
 'SP_grs1month',
 'SP_grs2month',
 'SP_grs3month',
 'SP_grs12month',
 'SP_weightedavgscore',
 'SP_percentilerank',
 'SP_expectedgrs',
 'clid',
 'SP_iscurrent',
 ]
df_merge = df_merge[cols]
# df_merge.columns.to_list()

In [99]:
'''
Check merging result
'''
# df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007032 entries, 5800 to 1651668
Data columns (total 24 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   prsku                      object  
 1   mkcname                    category
 2   clname                     category
 3   SP_suid                    category
 4   SP_iswaymore               int32   
 5   CR_percentilerank          float64 
 6   CR_biccontenttypename      category
 7   CR_hasbiccontenttype       int32   
 8   AT_biccontenttypename      category
 9   AT_Conversion_Rate_Change  float64 
 10  SP_addedtocart             int64   
 11  SP_placedorder             int64   
 12  SP_trafficcount            int64   
 13  SP_videocount              int64   
 14  MC_modulecount             float64 
 15  SP_grs1month               float64 
 16  SP_grs2month               float64 
 17  SP_grs3month               float64 
 18  SP_grs12month              float64 
 19  SP_weightedavgscor

In [79]:
df_merge.head(1)

,prsku,mkcname,clname,SP_suid,SP_iswaymore,CR_percentilerank,CR_biccontenttypename,CR_hasbiccontenttype,AT_biccontenttypename,AT_Conversion_Rate_Change,SP_addedtocart,SP_placedorder,SP_trafficcount,SP_videocount,MC_modulecount,SP_grs1month,SP_grs2month,SP_grs3month,SP_grs12month,SP_weightedavgscore,SP_percentilerank,SP_expectedgrs,clid,SP_iscurrent
5800,AACE1000,Rugs,Area Rugs,2373,0,NaN,Warranty & Guarantees,0,Warranty & Guarantees,0.0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,15,1


In [81]:
'''
Delete unncessary data
'''
df = df_merge
df = df.reset_index(drop=True)
del [col,
     cols,
     df_CR_AT,
     df_CR_MC_AT,
     df_merge,]
# %who_ls

In [86]:
'''
Save DataFrame to Excel file
'''
# df.to_csv('Merging_WayfairUSA.csv', index=False)